In [ ]:
from mx.progress import init_with_progress
init_with_progress()
from mx.prelude import *
u.set_debug()

In [2]:
import itertools
from mx import datasets as mxd, tasks as mxt, models as mxm, embeddings as mxe, layers as mxl
from mx.pipeline import Pipeline
from mx.visualizer import Visualizer

lengths = [
    (10, "10step", "(10 steps)"),
    (20000, "20kstep", "(20000 steps)"),
    (1_000_000, "1Mstep", "(1M steps)"),
]
sizes = [
    Box(
        name="tiny",
        desc="Tiny",
        n_layers=1,
        n_hidden=32,
        n_embd=8,
        n_heads=4,
        dropout=0.,
    ),
    Box(
        name="med",
        desc="Medium",
        n_layers=4,
        n_hidden=512,
        n_embd=128,
        n_heads=8,
        dropout=0.1,
    ),
    Box(
        name="big",
        desc="Full Size",
        n_layers=6,
        n_hidden=1024,
        n_embd=256,
        n_heads=12,
        dropout=0.1,
    ),
]
chunk_sizes = [
    # chunk   batch        name
    (1,    784*16,         "blind"),
    (28*2, 784*16//(28*2), "chunks"),
    (784,  16,             "fullimg"),
]
models = [
    lambda size: mxm.DebugMLP(
        n_hidden=size.n_hidden,
        dropout=size.dropout,
        name=f"debugmlp-{size.name}"
    ),
    lambda size: mxm.DecoderOnlyTransformer(
        n_layers=size.n_layers,
        n_hidden=size.n_hidden,
        n_heads=size.n_heads,
        dropout=size.dropout,
        name=f"transformer-{size.name}"
    ),
    lambda size: mxm.Resnet(
        n_layers=size.n_layers,
        n_hidden=size.n_hidden,
        dropout=size.dropout,
        name=f"resnet-{size.name}"
    ),
]
mnist_embeddings = [
    lambda size: mxe.DebugCodebook(
        n_embd=size.n_embd,
    ),
    lambda size: mxe.VectorCodebookMultidim(
        n_embd=size.n_embd,
    ),
    lambda size: mxe.VectorSinusoidalMultidim(
        n_embd=size.n_embd,
    ),
]

pipelines = []

cfgs = list(itertools.product(
    chunk_sizes,
    models,
    mnist_embeddings,
    sizes,
    lengths,
))

for (
    (chunk_size, batch_size, chunk_size_name),
    make_model,
    make_embedding,
    size,
    (length, length_name, length_desc),
) in cfgs:

    mxmodel = make_model(size)
    embedding = make_embedding(size)

    if chunk_size_name in ["blind", "chunked"] and (mxmodel.name.startswith("resnet") or mxmodel.name.startswith("debugmlp")):
        # Skip resnet blind and chunked tests, since this is just a batch size for them
        continue


    pipelines.append(
        Pipeline(
            name=f"{chunk_size_name}-{mxmodel.name}-{embedding.name}-{length_name}",
            desc=f"{mxmodel.desc} with {embedding.desc} {size.desc} {length_desc}",
            model=mxmodel,
            embedding=embedding,
            n_steps=length,
            batch_size=batch_size,
            viz_batch_size=10,
            dataset=mxd.MxMNIST(),
            task=mxt.VectorSequenceMSE(
                chunk_size=chunk_size,
            ),
        )
    )
pipelines = u.list_to_dict(pipelines)

# pipelines = u.list_to_dict([

#     Pipeline(
#         name="tiny-resnet",
#         desc="Tiny pipeline for debugging",
#         batch_size=8,
#         n_steps=100,
#         viz_batch_size=10,
#         dataset=mxd.MxMNIST(),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=24,
#         ),
#         embedding=mxe.VectorCodebookMultidim(
#             n_embd=8,
#         ),
#         model=mxm.Resnet(
#             n_layers=1,
#             n_hidden=64,
#             dropout=0.1,
#         ),
#     ),
#     Pipeline(
#         name="tiny-resnet-sin",
#         desc="Tiny pipeline for debugging",
#         batch_size=8,
#         n_steps=100,
#         viz_batch_size=10,
#         dataset=mxd.MxMNIST(),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=24,
#         ),
#         embedding=mxe.VectorSinusoidalMultidim(
#             n_embd=8,
#         ),
#         model=mxm.Resnet(
#             n_layers=1,
#             n_hidden=64,
#             dropout=0.1,
#         ),
#     ),
#     Pipeline(
#         name="tiny-resnet-sin-long",
#         desc="Tiny pipeline for debugging",
#         batch_size=8,
#         n_steps=1000000,
#         viz_batch_size=10,
#         dataset=mxd.MxMNIST(),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=24,
#         ),
#         embedding=mxe.VectorSinusoidalMultidim(
#             n_embd=8,
#         ),
#         model=mxm.Resnet(
#             n_layers=1,
#             n_hidden=64,
#             dropout=0.1,
#         ),
#     ),

#     Pipeline(
#         name="bavecdo",
#         desc="BVH hand-vec regression w decoder-only transformer",
#         batch_size=32,
#         n_steps=10000,
#         model=mxm.DecoderOnlyTransformer(
#             n_layers=4,
#             n_heads=16,
#             n_hidden=1024,
#         ),
#         embedding=mxe.AngleVectorSequence(
#             n_embd=128,
#             n_repeats=12,
#         ),
#         task=mxt.VectorSequenceAngleMSE(
#             chunk_size=500,
#         ),
#         dataset=mxd.BvhDataset(
#             recluster=False,
#             decimate=False,
#         ),
#     ),

#     Pipeline(
#         name="mnext-full",
#         desc="MNIST pixel regression w decoder-only transformer",
#         batch_size=16,
#         viz_batch_size=10,
#         n_steps=20000,
#         model=mxm.DecoderOnlyTransformer(
#             n_layers=3,
#             n_heads=12,
#             n_hidden=128,
#             dropout=0.1,
#         ),
#         embedding=mxe.VectorCodebookMultidim(
#             n_embd=64,
#         ),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=784,
#         ),
#         dataset=mxd.MxMNIST(),
#     ),
#     Pipeline(
#         name="mnext-32",
#         desc="MNIST pixel regression w decoder-only transformer",
#         batch_size=16,
#         viz_batch_size=10,
#         n_steps=20000,
#         model=mxm.DecoderOnlyTransformer(
#             n_layers=3,
#             n_heads=12,
#             n_hidden=128,
#             dropout=0.1,
#         ),
#         embedding=mxe.VectorCodebookMultidim(
#             n_embd=64,
#         ),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=32,
#         ),
#         dataset=mxd.MxMNIST(),
#     ),
#     Pipeline(
#         name="msinext",
#         desc="MNIST pixel regression w decoder-only transformer (sinusoidal embd)",
#         batch_size=32,
#         viz_batch_size=8,
#         n_steps=10000,
#         model=mxm.DecoderOnlyTransformer(
#             n_layers=4,
#             n_heads=16,
#             n_hidden=1024,
#         ),
#         embedding=mxe.VectorSinusoidalMultidim(
#             n_embd=128,
#         ),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=512,
#         ),
#         dataset=mxd.MxMNIST(),
#     ),
    
#     Pipeline(
#         name="mrandom",
#         desc="MNIST random-order pixel regression w decoder-only transformer",
#         batch_size=32,
#         viz_batch_size=8,
#         n_steps=10000,
#         model=mxm.DecoderOnlyTransformer(
#             n_layers=4,
#             n_heads=16,
#             n_hidden=1024,
#         ),
#         embedding=mxe.VectorCodebookMultidim(
#             n_embd=128,
#         ),
#         task=mxt.VectorSequenceMSE(
#             chunk_size=512,
#         ),
#         dataset=mxd.MxMNIST(),
#     ),
# ])

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


NameError: name 'Box' is not defined

In [ ]:
from mx import train

pipeline = pipelines[
    list(pipelines.keys())[
        int(input(
            "Choose pipeline:\n"
            + "\n".join([
                f"    ({i: <3}): {p.name}"
                for i, p in enumerate(pipelines.values())
            ])
            + "    choice: "
        ))
    ]
]

prompt = """
Start a new run?
          [ Y = yes ]       
          [ N = no  ]       
(default) [ D = dev ] choice: """


run_config = input(prompt).strip().lower()
if run_config.startswith("y"):
    do_new_run = True
    run_name = u.random_run_name()
    print(f"Starting new run {run_name}")
elif run_config.startswith("n"):
    do_new_run = False
    run_name = input("Run name: ")
    u.set_run_name(run_name)
    print(f"Loading run {run_name}")
else:
    do_new_run = True
    run_name = "dev"
    u.set_run_name(run_name)
    print(f"Using run_name = {run_name}")

output_dir = pipeline.output_dir()

if pipeline.use_float16:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

model = pipeline.make_or_load_model(force_new=do_new_run)
loss_fn = pipeline.make_loss_fn()
data = pipeline.make_train_data()
predict_fn = pipeline.task.make_predict_fn(model)
vizs = pipeline.dataset.get_visualizations(
    viz_batch_size=pipeline.viz_batch_size,
    task_specific_predict_fn=predict_fn,
)
vizr = Visualizer(
    visualizations=vizs,
    output_dir=output_dir,
    configs={},
)
train_loop = train.make_train_loop(
    model=model,
    loss_fn=loss_fn,
    data=data,
    run_name=run_name,
    output_dir=output_dir,
    vizr=vizr,
    log_interval="never",
)

In [ ]:
from mx.progress import create_progress_manager

if do_new_run:
    train_loop()